<center> <h1> 机器作诗进阶（2）：LSTM </h1> </center>  

# RNN存在的问题

> * 如果让RNN完成上面的题目，它可以很容易的猜出（1），但是猜不出（2）
* 这是因为RNN无法处理“长距离依赖”（Long-Term Dependencies）
* RNN的这一短板是由于其算法导致：在训练RNN模型时，每步的梯度随距离会明显减弱，因此当距离过长时，迅速减小的梯度无法将之前的信息传递过去


# 改进思路
> * 原始RNN在隐藏层只有一个状态，即上一时刻的状态，它对于短期的输入非常敏感。
* 加入我们再增加一个状态，让它来保存“长期信息”，问题是不是就可以解决啦？


# RNN的进阶：LSTM

> * LSTM （Long Short Term Memory Network），也叫长短期记忆网络。
* LSTM是RNN的一个优秀的变种模型，能很好的处理“长距离依赖”问题


# LSTM的改进

> 相比于传统的RNN，LSTM主要有两方面的改进：
> * 在输入中增加了长期状态（$c_{t-1}$）
> * 内部对信息的处理更加复杂：多过程（红色圈圈）+多层网络（黄色方块）


# 预备知识


### $\tanh$ 变换
$\tanh$ 也是常用的非线性激活函数，可以将一个实数映射到 $(-1,1)$ 的区间，其数学表达式如下：
$$
\tanh x = \frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}
$$
与 sigmoid 不同的是，$\tanh$ 是0均值的,  而且 sigmoid 函数在输入处于0附近时，函数值变化比 $\tanh$ 敏感，一旦接近或者超出区间就失去敏感性，处于饱和状态

### sigmoid 变换

sigmoid 函数也叫 Logistic 函数，是神经网络中常用的非线性激活函数，可以将一个实数映射到 $(0,1)$ 的区间，其数学表达式如下：
$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$


# LSTM的核心
> 使用三个控制开关，控制信息的输入和输出


# LSTM结构详解

### 遗忘门（Forgotten Gate）
> * LSTM的第一个环节是进入遗忘门，所谓“遗忘”，是指数据通过这个结构后要“过滤”一部分信息
* 这部分的输入为$h_{t-1}$和$x_t$，经过一个sigmoid layer后输出一个$0$和$1$之间的数，越接近于$1$表示保留的信息越多


# LSTM结构详解


### 输入门（Input Gate）
> * 输入门将决定对下一时刻的状态加入多少“新信息”
* 它由两层结构组成：前面的sigmoid layer将决定我们更新数据中的哪几个值，后面的tanh layer则会对数据中所有值输出更新值向量$\tilde{C_t}$。

> 经过上面两步，可以计算当前时刻的长期状态$C_t$


# LSTM结构详解

### 输出门（Output Gate）
> * 输出门将决定单元的输出值$h_t$。
* 它同样由两层结构组成：前面的sigmoid layer将决定我们输出数据中的哪部分值，同时长期状态$C_t$将通过tanh layer转换至$-1$至$1$之间，最后的输出结果为两者的乘积

# 数据处理：与RNN相同

## 数据介绍

> `data/poems_clean.txt`
* 格式：标题:诗（每句之间以空格隔开）
* 例子：
    >> 静夜思:床前明月光 疑是地上霜 举头望明月 低头思故乡  
    >> 春望:国破山河在 城春草木深 感时花溅泪 恨别鸟惊心 烽火连三月 家书抵万金 白头搔更短 浑欲不胜簪 

## 数据的读入与展示


In [1]:
import string
import numpy as np

f = open('data/poems_clean.txt', "r", encoding='utf-8')
poems = []
for line in f.readlines():
    title, poem = line.split(':')
    poem = poem.replace(' ', '') #将空格去掉
    poem = poem.replace('\n', '') #将换行符去掉
    poems.append(list(poem))
    
print(poems[0][:])

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']


## 数据整理：文字编码

> 读入数据并使用`keras`中的`Tokenizer`为我们的语料库建立词典，给每个字分配一个索引。

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(poems)
vocab_size = len(tokenizer.word_index) + 1 #加上停止词0
poems_digit = tokenizer.texts_to_sequences(poems)
#为了将所有的诗放在一个M*N的np.array中，将每一首诗补0到同样的长度
poems_digit = pad_sequences(poems_digit, maxlen=50, padding='post')

Using TensorFlow backend.


## 数据整理：生成$X$和$Y$

### （1）数据补全

In [3]:
print("原始诗歌")
print(poems[3864])
print("\n")
print("编码+补全后的结果")
print(poems_digit[3864])

原始诗歌
['床', '前', '明', '月', '光', '疑', '是', '地', '上', '霜', '举', '头', '望', '明', '月', '低', '头', '思', '故', '乡']


编码+补全后的结果
[532  72  53  13 140 429  44 113  15 202 688 128 106  53  13 502 128  75
 134 169   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


### （2）生成$X$和$Y$

In [4]:
X = poems_digit[:, :-1]
Y = poems_digit[:, 1:]
print("X示例", "\t", "Y示例")

for i in range(10):
    print(X[0][i], "\t", Y[0][i])
    
print("...", "\t", "...")

X示例 	 Y示例
42 	 180
180 	 401
401 	 1143
1143 	 671
671 	 9
9 	 331
331 	 130
130 	 58
58 	 84
84 	 177
... 	 ...


### （3）把Y变成One-Hot向量

In [5]:
from keras.utils import to_categorical
Y = to_categorical(Y, num_classes=vocab_size)
print(Y.shape)

(24117, 49, 5546)


# 构建LSTM的模型

In [6]:
# from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Activation, BatchNormalization
from keras import Model

hidden_size1 = 128
hidden_size2 = 64

inp = Input(shape=(49,))

# Encoder
x = Embedding(vocab_size, hidden_size1, input_length=49, mask_zero=True)(inp)
x = LSTM(hidden_size2, return_sequences=True)(x)

# prediction
x = Dense(vocab_size)(x)
pred = Activation('softmax')(x)

model = Model(inp, pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 49)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 49, 128)           709888    
_________________________________________________________________
lstm_1 (LSTM)                (None, 49, 64)            49408     
_________________________________________________________________
dense_1 (Dense)              (None, 49, 5546)          360490    
_________________________________________________________________
activation_1 (Activation)    (None, 49, 5546)          0         
Total params: 1,119,786
Trainable params: 1,119,786
Non-trainable params: 0
_________________________________________________________________


# 让我们来数数参数

### Embedding层
我们一共有5546个字，每个字嵌入到一个128维的空间中，所以参数个数为：$5546\times 128=70988$。 

### LSTM层
第一、参考之前LSTM的介绍可以知道，需要参数估计的非线性变幻主要涉及到：$f_t$, $i_t$, $\tilde C_t$, 还有 $o_t$。每个非线性变化所消耗的参数一样。背后主要的原因是：TF要求$h_t$和$c_t$的维度一样（理论上完全可以不一样）。

第二、以$f_t$为例，它作用在$(h_{t-1},x_{t})$上面。由于$h_{t-1}$和$x_{t}$分别是一个64维和128维的向量。加上截距项，需要64+128+1=193个参数。这些参数应用到遗忘门，帮助$C_t$状态更新的时候，需要消耗：193*64=12352个参数。

第三，因为，$f_t$, $i_t$, $\tilde C_t$, 还有$o_t$一共4个非线性变换，而每一个变换所消耗的参数都是12352，因此，最终所需的所有参数是：$12352\times 4=49408$。

### dense层
这是一个5546类的分类问题，输入就是h维度+常数项，所以参数个数为：$(64+1)\times 5546=360490$。

# 模型训练 

In [10]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
model.fit(X, Y, epochs=10, batch_size=128, validation_split=0.2)

# 应用模型作诗

In [8]:
poem_incomplete = '熊****大****很****帅****'
poem_index = []
poem_text = ''
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    
    if  current_word != '*':
        index = tokenizer.word_index[current_word]
        
    else:
        x = np.expand_dims(poem_index, axis=0)
        x = pad_sequences(x, maxlen=49, padding='post')
        y = model.predict(x)[0, i]
        
        y[0] = 0            #去掉停止词
        index = y.argmax()
        current_word = tokenizer.index_word[index]
        
        


    poem_index.append(index)
    poem_text = poem_text + current_word
        
poem_text = poem_text[0:]
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

熊颀颀颀坑
大人不不不
很不不不不
帅不不不不


# 思考问题：RNN vs. 时间序列模型